In [197]:
import requests
import json
import datetime

class BlockstreamAPI:
    def __init__(self, base_url="https://blockstream.info/api"):
        self.base_url = base_url

    # Blocks
    def get_block(self, block_hash):
        url = f"{self.base_url}/block/{block_hash}"
        return self._make_request(url)

    def get_block_header(self, block_hash):
        url = f"{self.base_url}/block/{block_hash}/header"
        return self._make_request(url)

    def get_block_status(self, block_hash):
        url = f"{self.base_url}/block/{block_hash}/status"
        return self._make_request(url)

    def get_block_transactions(self, block_hash, start_index=0):
        url = f"{self.base_url}/block/{block_hash}/txs/{start_index}"
        return self._make_request(url)

    def get_block_txids(self, block_hash):
        url = f"{self.base_url}/block/{block_hash}/txids"
        return self._make_request(url)

    def get_block_txid_at_index(self, block_hash, index):
        url = f"{self.base_url}/block/{block_hash}/txid/{index}"
        return self._make_request(url)

    def get_block_raw(self, block_hash):
        url = f"{self.base_url}/block/{block_hash}/raw"
        return self._make_request(url)

    def get_block_hash_by_height(self, height):
        url = f"{self.base_url}/block-height/{height}"
        return self._make_request(url).decode('utf-8')

    def get_latest_blocks(self, start_height=None):
        url = f"{self.base_url}/blocks"
        if start_height is not None:
            url = f"{self.base_url}/blocks/{start_height}"
        return self._make_request(url)

    def get_latest_block_height(self):
        url = f"{self.base_url}/blocks/tip/height"
        return self._make_request(url)

    def get_latest_block_hash(self):
        url = f"{self.base_url}/blocks/tip/hash"
        return self._make_request(url)

    def get_block_info_by_height(self, height):
        block_hash = self.get_block_hash_by_height(height)
        return self.get_block(block_hash)

        # Txs
    def get_transaction(self, txid):
        url = f"{self.base_url}/tx/{txid}"
        return self._make_request(url)

    def get_transaction_status(self, txid):
        url = f"{self.base_url}/tx/{txid}/status"
        return self._make_request(url)

    def get_transaction_hex(self, txid):
        url = f"{self.base_url}/tx/{txid}/hex"
        return self._make_request(url)

    def get_transaction_raw(self, txid):
        url = f"{self.base_url}/tx/{txid}/raw"
        return self._make_request(url)

    def get_transaction_merkleblock_proof(self, txid):
        url = f"{self.base_url}/tx/{txid}/merkleblock-proof"
        return self._make_request(url)

    def get_transaction_merkle_proof(self, txid):
        url = f"{self.base_url}/tx/{txid}/merkle-proof"
        return self._make_request(url)

    def get_transaction_outspend(self, txid, vout):
        url = f"{self.base_url}/tx/{txid}/outspend/{vout}"
        return self._make_request(url)

    def get_transaction_outspends(self, txid):
        url = f"{self.base_url}/tx/{txid}/outspends"
        return self._make_request(url)

    def _make_request(self, url):
        response = requests.get(url)

        if response.status_code == 200:
            try:
                return json.loads(response.content)
            except:
                # print(f"Json decode error in {response.headers}.")
                return response.content
        else:
            response.raise_for_status()

In [198]:
api = BlockstreamAPI()

In [199]:
bheight = api.get_latest_block_height()
print("bheight", bheight)
bhash = api.get_block_hash_by_height(bheight)
print("bhash", bhash)

bheight 844939
bhash 000000000000000000005f850a0dde4a5f8f611e83d9ed578f38fbe068b6f8cf


In [200]:
def ts_to_datetime(ts):
    return datetime.datetime.utcfromtimestamp(ts)

def block_info_to_datetime(block_info):
    return ts_to_datetime(block_info['timestamp'])

In [201]:
def binary_search(getfn, target, left, right):
    while right - left > 1:
        middle = (left + right) // 2
        if getfn(middle) >= target:
            right = middle
        else:
            left = middle
    return left

In [202]:
arr = [1, 3, 5, 7, 9, 11, 13, 15]
def getarr(i):
    return arr[i]
target = 9
for i, e in enumerate(arr):
    print(i, e)

print(f"binary_search(getarr, {target}, 0, {len(arr)})", binary_search(getarr, target, 0, len(arr)))

0 1
1 3
2 5
3 7
4 9
5 11
6 13
7 15
binary_search(getarr, 9, 0, 8) 3


In [203]:
dt_start = datetime.datetime(2024, 3, 13, 22, 59, 0)
dt_end = datetime.datetime(2024, 3, 18, 18, 34, 0)

In [204]:
def block_date_by_height(height):
    return block_info_to_datetime(api.get_block_info_by_height(height))

In [205]:
search_start = 830000
search_end = 840000
print(block_date_by_height(search_start))
print(block_date_by_height(search_end))

2024-02-11 18:17:37


KeyboardInterrupt: 

In [ ]:
start_block = binary_search(block_date_by_height, dt_start, search_start, search_end)
start_block

In [ ]:
end_block = binary_search(block_date_by_height, dt_end, search_start, search_end)
end_block = end_block + 1
end_block

In [ ]:
end_block - start_block

In [ ]:
print(block_date_by_height(start_block))
print(block_date_by_height(end_block))

In [ ]:
txs = []
for bn in range(start_block, end_block):
    block_info = api.get_block_info_by_height(bn)
    block_hash = block_info['id']
    tx_count = block_info['tx_count']
    for ind in range(0, tx_count, 25):
        txs.extend(api.get_block_transactions(block_hash, ind))